In [1]:
import pyqg
import numpy as np
import matplotlib.pyplot as plt

# Run four models -- [2,3] layers x [64, 96] resolution

In [2]:
year = 24*60*60*360.

models = [[],[]]

for i, model_class in enumerate([pyqg.LayeredModel, pyqg.QGModel]):
    for nx in [96,64]:
        model = model_class(nx=nx, tmax=10*year, twrite=10000, tavestart=5*year, dt=3600)
        model.run()
        models[i].append(model)

INFO:  Logger initialized
INFO: Step: 10000, Time: 3.60e+07, KE: 6.24e-03, CFL: 0.236
INFO: Step: 20000, Time: 7.20e+07, KE: 1.46e-02, CFL: 0.275
INFO: Step: 30000, Time: 1.08e+08, KE: 1.62e-02, CFL: 0.317
INFO: Step: 40000, Time: 1.44e+08, KE: 1.73e-02, CFL: 0.316
INFO: Step: 50000, Time: 1.80e+08, KE: 1.90e-02, CFL: 0.311
INFO: Step: 60000, Time: 2.16e+08, KE: 1.38e-02, CFL: 0.323
INFO: Step: 70000, Time: 2.52e+08, KE: 1.81e-02, CFL: 0.351
INFO: Step: 80000, Time: 2.88e+08, KE: 1.78e-02, CFL: 0.344
INFO:  Logger initialized
INFO: Step: 10000, Time: 3.60e+07, KE: 6.41e-03, CFL: 0.135
INFO: Step: 20000, Time: 7.20e+07, KE: 1.23e-02, CFL: 0.170
INFO: Step: 30000, Time: 1.08e+08, KE: 1.24e-02, CFL: 0.164
INFO: Step: 40000, Time: 1.44e+08, KE: 1.45e-02, CFL: 0.213
INFO: Step: 50000, Time: 1.80e+08, KE: 1.50e-02, CFL: 0.176
INFO: Step: 60000, Time: 2.16e+08, KE: 1.28e-02, CFL: 0.160
INFO: Step: 70000, Time: 2.52e+08, KE: 1.51e-02, CFL: 0.192
INFO: Step: 80000, Time: 2.88e+08, KE: 1.36e-02,

# Check diagnostics for proper behavior / normalization

For all diagnostics, the lower-resolution model should have smaller extremes. However, lower-resolution model diagnostics should still have the same order of magnitude.

In [30]:
for models_by_res in models:
    print(models_by_res[0].__class__.__name__)
    diagnostics = models_by_res[0].diagnostics.keys()
    for diagnostic in diagnostics:
        vals = []
        for m in models_by_res:
            max_val = np.abs(m.get_diagnostic(diagnostic)).max()
            vals.append(max_val)
        max_hi, max_lo = vals 
        if max_lo == 0:
            assert max_hi == 0
        else:
            print(f"   {diagnostic} ratio = {max_hi/max_lo:.2f}")
            assert max_hi/max_lo < 2.5
            assert max_hi/max_lo > 0.9

LayeredModel
   Ensspec ratio = 1.09
   KEspec ratio = 1.18
   EKEdiss ratio = 1.29
   KEfrictionspec ratio = 1.25
   EKE ratio = 1.24
   Dissspec ratio = 1.24
   ENSDissspec ratio = 2.44
   entspec ratio = 1.35
   KEspec_modal ratio = 1.17
   PEspec_modal ratio = 1.16
   APEspec ratio = 1.14
   KEflux_div ratio = 1.23
   APEflux_div ratio = 1.15
   APEgenspec ratio = 1.16
   ENSflux ratio = 1.15
   ENSgenspec ratio = 1.15
   ENSfrictionspec ratio = 1.06
QGModel
   Ensspec ratio = 1.18
   KEspec ratio = 1.20
   EKEdiss ratio = 1.36
   KEfrictionspec ratio = 1.35
   EKE ratio = 1.21
   Dissspec ratio = 1.13
   ENSDissspec ratio = 2.39
   entspec ratio = 0.90
   APEflux ratio = 1.50
   KEflux ratio = 1.36
   APEgenspec ratio = 1.29
   APEgen ratio = 1.22


The same should roughly hold true for instantaneous diagnostics, though with a bit more randomness:

In [35]:
for models_by_res in models:
    print(models_by_res[0].__class__.__name__)
    diagnostics = models_by_res[0].diagnostics.keys()
    for diagnostic in diagnostics:
        vals = []
        for m in models_by_res:
            max_val = np.abs(m.diagnostics[diagnostic]['function'](m)).max()
            vals.append(max_val)
        max_hi, max_lo = vals 
        if max_lo == 0:
            assert max_hi == 0
        else:
            print(f"   {diagnostic} ratio = {max_hi/max_lo:.2f}")
            assert max_hi/max_lo < 3
            assert max_hi/max_lo > 0.8

LayeredModel
   Ensspec ratio = 1.43
   KEspec ratio = 1.10
   EKEdiss ratio = 1.63
   KEfrictionspec ratio = 1.31
   EKE ratio = 1.40
   Dissspec ratio = 0.97
   ENSDissspec ratio = 2.01
   entspec ratio = 0.99
   KEspec_modal ratio = 1.18
   PEspec_modal ratio = 2.03
   APEspec ratio = 1.88
   KEflux_div ratio = 1.86
   APEflux_div ratio = 0.90
   APEgenspec ratio = 1.11
   ENSflux ratio = 1.83
   ENSgenspec ratio = 1.12
   ENSfrictionspec ratio = 1.27
QGModel
   Ensspec ratio = 1.39
   KEspec ratio = 1.11
   EKEdiss ratio = 1.49
   KEfrictionspec ratio = 1.39
   EKE ratio = 1.18
   Dissspec ratio = 1.35
   ENSDissspec ratio = 2.94
   entspec ratio = 0.94
   APEflux ratio = 1.87
   KEflux ratio = 1.58
   APEgenspec ratio = 1.51
   APEgen ratio = 1.33
